# Sampling

In [3]:
import os
import json
import random
import shutil
import tqdm
import cv2


In [4]:
label_kr = {
    'c_1' :0, #'종이',
    'c_2_01': 1, #'종이팩',
    'c_2_02': 2, #'종이컵',
    'c_3': 3, #'캔류',
    'c_4_01_02': 4, #'재사용 유리(소주병 + 맥주병)',
    'c_4_02_01_02': 5, #'갈색 유리',
    'c_4_02_02_02': 6, #'녹색 유리',
    'c_4_02_03_02': 7, #'백색 유리',
    'c_4_03': 8, #'기타 유리',
    'c_5_02': 9, #'페트',
    'c_6': 10, #'플라스틱',
    'c_7': 11, #'비닐',
    ## 이 위에는 재활용
    ## 밑에는 일반쓰레기
    'c_1_01': 12, #'종이 + 이물질',
    'c_2_02_01': 13, #'종이컵 + 이물질',
    'c_3_01': 14, #'캔 + 이물질',
    'c_4_03_01': 15, #'기타 유리 + 이물질',
    'c_5_01_01': 16, #'페트 + 이물질 + 다중포장재',
    'c_5_02_01' : 17, #'페트+이물질',
    'c_6_01': 18, #'플라스틱 + 이물질',
    'c_7_01': 19, #'비닐 + 이물질',

    # 다중포장재
    'c_4_01_01': 20, #'재사용 유리(소주병+맥주병) + 다중 포장재',
    'c_4_02_01_01': 21, #'갈색 유리 + 다중포장재',
    'c_4_02_02_01': 22, #'녹색 유리 + 다중포장재',
    'c_4_02_03_01': 23, #'백색 유리 + 다중포장재',
    'c_5_01': 24, #'페트 + 다중포장재',
    'c_8_01': 25, #'흰색 스티로폼',
    'c_8_02': 26, #'컬러 스티로폼',
    'c_8_01_01': 27, #'스티로폼 + 이물질',
    'c_9': 28, #'건전지'
}

# 무결성 검사
존재하는 이미지만 json파일 추출

In [5]:
images_path = './unzip_image'
json_path = './unzip_label'

to_image_path = './image_hub'
to_json_path = './label_hub'

images_list = os.listdir(images_path)
json_list = os.listdir(json_path)

print('이미지 형식 jpg로 통일')
for file_name in tqdm.tqdm(images_list):
    if file_name.endswith('.JPG'):
        new_file_name = file_name.replace('.JPG', '.jpg')
        old_file_path = os.path.join(images_path, file_name)
        new_file_path = os.path.join(images_path, new_file_name)
        os.rename(old_file_path, new_file_path)

for file_list in [images_list,json_list]:
    for idx , file_name in tqdm.tqdm(enumerate(file_list)):
        file_list[idx] = os.path.splitext(file_name)[0]

cnt = 0
for file_name in tqdm.tqdm(images_list):
    if file_name not in json_list: 
        os.remove(os.path.join(images_path,file_name+'.jpg'))
    else:
        cnt += 1
        shutil.move(os.path.join(images_path,file_name+'.jpg'),os.path.join(to_image_path,file_name+'.jpg'))
        shutil.move(os.path.join(json_path,file_name+'.json'),os.path.join(to_json_path,file_name+'.json'))

print(f'옮긴 데이터 개수 : {cnt}게')

151453it [00:00, 584277.49it/s]
20088it [00:00, 559660.30it/s]
100%|██████████| 20088/20088 [00:41<00:00, 484.59it/s]


이미지 개수 : 151453
json 개수 : 20078


# JSON to TXT

In [4]:
# Json to txt
label_change = {
    0:0 , 1:1 , 2:2 , 3:3 , 4:4 , 5:4 , 6:4 , 7:4 , 8:4 , 9:5 , 10:6 , 11:7 , 12:0 , 13:1 , 14:3 , 15:4 , 16:9 , 17:5 , 18:6 ,
    19:7 , 20:8 , 21:8, 22:8 , 23:8 , 24:9 , 25:10 , 26: 10 , 27:10 , 28:11
}

from_label_path = './label_hub'
from_image_path = './image_hub'

json_list = os.listdir(from_label_path)

for file_name in tqdm.tqdm(json_list):
    try:
        image_name = os.path.splitext(file_name)[0]+'.jpg'
        image = cv2.imread(os.path.join(from_image_path,image_name))
        h , w , _ = image.shape  # 세로,가로
        resized_image = cv2.resize(image,(640,640))
        cv2.imwrite(os.path.join(from_image_path,image_name),resized_image)

        with open(os.path.join(from_label_path,file_name), 'r',encoding="UTF-8") as file:
            data = json.load(file)
            
        extracted_objects = []      
        for dictionary in data["objects"]:                 
            class_name = label_change[label_kr[dictionary["class_name"]]]
            coord_x = dictionary["annotation"]["coord"]["x"]
            coord_y = dictionary["annotation"]["coord"]["y"]
            coord_width = dictionary["annotation"]["coord"]["width"]
            coord_height = dictionary["annotation"]["coord"]["height"]
            extracted_objects.append((class_name, (coord_x+coord_width/2)/w,
                                    (coord_y+coord_height/2)/h,coord_width/w,coord_height/h))
        # 추출한 값을 텍스트 파일로 저장
        text_file_name = os.path.splitext(file_name)[0] + '.txt'
        text_file_path = os.path.join(from_label_path, text_file_name)
        with open(text_file_path, 'w') as file:
            for obj in extracted_objects:
                file.write(' '.join(map(str, obj)) + '\n')
        os.remove(os.path.join(from_label_path,file_name))

    except:
        print(file_name)
        os.remove(os.path.join(from_image_path,os.path.splitext(file_name)[0]+'.jpg'))
        os.remove(os.path.join(from_label_path,file_name))


    

  0%|          | 2/114960 [00:00<2:41:03, 11.90it/s]

C_20220817_000825.json


 62%|██████▏   | 71787/114960 [2:18:53<1:04:11, 11.21it/s]

C_20220920_000122.json


100%|██████████| 114960/114960 [3:44:05<00:00,  8.55it/s] 


# 레이블 분포

In [5]:
label_path = './label_hub'
label_list = os.listdir(label_path)
print(f'final C 데이터 셋 개수 : {len(label_list)}')

label_count = [0 for _ in range(12)]

for label_name in tqdm.tqdm(label_list):
    temp_list = [0 for _ in range(12)]
    with open(os.path.join(label_path,label_name), 'r',encoding="UTF-8") as file:
        for line in file:
            temp_list[int(line.split()[0])] += 1
    for i in range(12):
            label_count[i] += temp_list[i]
print(label_count)


final C 데이터 셋 개수 : 151453


100%|██████████| 151453/151453 [10:32<00:00, 239.48it/s]

[34041, 9107, 5290, 28135, 18302, 10886, 47216, 44072, 19299, 21502, 11330, 6499]


# Uniform 분포

In [33]:
# 폴더 명 확인인

from_image_path = './image_hub'
from_label_path = './label_hub'
to_image_path = './Final_C_data/train/images'
to_label_path = './Final_C_data/train/labels'

max_val = 5700
min_val = 5500
seed = 1234

AB_label_count=[0 for _ in range(12)]


special = [1,2,11]
label_list = os.listdir(from_label_path)
random.Random(seed).shuffle(label_list) # random shuffling
for label_name in tqdm.tqdm(label_list):
    image_name = os.path.splitext(label_name)[0] + '.jpg'
    temp_list = [0 for _ in range(12)]
    with open(os.path.join(from_label_path,label_name), 'r',encoding="UTF-8") as file:
        for line in file:
            temp_list[int(line.split()[0])] += 1

    if sum(temp_list[idx] for idx in special) and all((AB_label_count[idx] + temp_list[idx] < min_val) for idx in range(12)):
        for i in range(12):
            AB_label_count[i] += temp_list[i]
        shutil.move(os.path.join(from_label_path,label_name) , os.path.join(to_label_path,label_name))
        shutil.copy(os.path.join(from_image_path,image_name) , os.path.join(to_image_path,image_name))
    
print(AB_label_count)


break_x = False
for patience in range(3):
    label_list = os.listdir(from_label_path)
    random.Random(seed).shuffle(label_list) # random shuffling

    for label_name in tqdm.tqdm(label_list):
        image_name = os.path.splitext(label_name)[0] + '.jpg'
        if all(AB_label_count[i]<min_val for i in range(12)):
            temp_list = [0 for _ in range(12)]
            with open(os.path.join(from_label_path,label_name), 'r',encoding="UTF-8") as file:
                for line in file:
                    temp_list[int(line.split()[0])] += 1
            for i in range(12):
                AB_label_count[i] += temp_list[i]
            shutil.move(os.path.join(from_label_path,label_name) , os.path.join(to_label_path,label_name))
            shutil.copy(os.path.join(from_image_path,image_name) , os.path.join(to_image_path,image_name))
    

        elif any(AB_label_count[i]<min_val for i in range(12)):
            need_list = [1 if x < min_val else 0 for x in AB_label_count]
            temp_list = [0 for _ in range(12)]
            with open(os.path.join(from_label_path,label_name), 'r',encoding="UTF-8") as file:
                for line in file:
                    temp_list[int(line.split()[0])] += 1

            go_x = True
            cnt = 0
            for i in range(12):
                if need_list[i]:
                    if temp_list[i] > 0:
                        cnt += 1
                else:
                    if temp_list[i] > 2*patience or (AB_label_count[i] > max_val + 2*(patience**2)) :
                        go_x = False
                        break

                        
            if go_x and cnt:
                for i in range(12):
                    AB_label_count[i] += temp_list[i]
                
                shutil.move(os.path.join(from_label_path,label_name) , os.path.join(to_label_path,label_name))
                shutil.copy(os.path.join(from_image_path,image_name) , os.path.join(to_image_path,image_name))
            
        if all(AB_label_count[i]>=min_val for i in range(12)):
            break_x = True
            break
    if break_x:
        break   
    print(AB_label_count) # samplinge된 클래스별 개수
print(AB_label_count) 

for path in [to_image_path,to_label_path]:
    moved_lsit = os.listdir(path)
    print(len(moved_lsit))

100%|██████████| 144757/144757 [11:17<00:00, 213.60it/s]


[3083, 5499, 4121, 4256, 173, 166, 3638, 4473, 2023, 2192, 241, 5267]


100%|██████████| 132338/132338 [11:31<00:00, 191.25it/s]


[5500, 5500, 4121, 5500, 5500, 5500, 5501, 5500, 5500, 5500, 5501, 5267]


100%|██████████| 107428/107428 [01:14<00:00, 1441.29it/s]


[5545, 5703, 4211, 5558, 5503, 5500, 5556, 5551, 5551, 5556, 5504, 5382]


100%|██████████| 107239/107239 [00:20<00:00, 5127.51it/s]


[5545, 5709, 4214, 5558, 5503, 5500, 5557, 5555, 5553, 5558, 5504, 5387]
[5545, 5709, 4214, 5558, 5503, 5500, 5557, 5555, 5553, 5558, 5504, 5387]
37524
37524


In [32]:
# 위에 코드 재실행할 때 사용
json_list = os.listdir(to_label_path)
cnt = 0
for file_name in json_list:
    #shutil.copy(os.path.join(to_path,file_name) , os.path.join(AB_path,file_name))
    shutil.move(os.path.join(to_label_path,file_name) , os.path.join(from_label_path,file_name))
    cnt += 1
print(cnt)

0


In [34]:
tr_vl = ['train','val','test']


for t_v in tr_vl:
    from_path = os.path.join('./Final_C_data',t_v)
    image_path = os.path.join(from_path,'images')
    label_path = os.path.join(from_path,'labels')

    image_list = os.listdir(image_path)
    label_list = os.listdir(label_path)

    label_count=[0 for _ in range(12)]

    for file_name in tqdm.tqdm(label_list):
        with open(os.path.join(label_path,file_name), 'r',encoding="UTF-8") as file:
            for line in file:
                label_count[int(line.split()[0])] += 1
                
    image_list = [os.path.splitext(v)[0] for v in image_list]
    label_list = [os.path.splitext(v)[0] for v in label_list]
    image_label_list = set(image_list+label_list)

    print(f'{t_v} : 이미지 개수 : {len(image_list)} / txt 개수 : {len(label_list)} / 이미지,txt쌍 개수 : {len(image_label_list)}')
    print(label_count)
    print()


100%|██████████| 37524/37524 [01:58<00:00, 317.66it/s]


train : 이미지 개수 : 37524 / txt 개수 : 37524 / 이미지,txt쌍 개수 : 37524
[5545, 5709, 4214, 5558, 5503, 5500, 5557, 5555, 5553, 5558, 5504, 5387]



100%|██████████| 3414/3414 [00:16<00:00, 205.60it/s]


val : 이미지 개수 : 3414 / txt 개수 : 3414 / 이미지,txt쌍 개수 : 3414
[500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500]



100%|██████████| 3282/3282 [00:14<00:00, 231.11it/s]


test : 이미지 개수 : 3282 / txt 개수 : 3282 / 이미지,txt쌍 개수 : 3282
[500, 500, 500, 501, 500, 500, 500, 502, 500, 500, 500, 500]

